The interest of this project is attempting to segment customers on the basis of their spending on different types of products (wine, fruits, meat, etc.). Regression methods will be used to determine the significant features in each segment, which will help us determine the group at which marketing campaigns for a specific type of product should be targeted.

In [ ]:
# System setup
rm(list = ls())
knitr::opts_chunk$set(echo = TRUE, cache=TRUE)
if(!require('pacman')) {
  install.packages('pacman')
}
pacman::p_load(caret,dplyr,rsample, recipes, ggplot2, nnet, gam, tidyverse, 
               GGally, corrplot, FNN, gmodels, earth, kernlab, vip, DiagrammeR,rsvg,
               rpart,rpart.plot,ranger, h2o, gbm, xgboost, e1071, party, partykit, Metrics)

# 1. Exploratory Data Analysis and Data Mutation
Load the data.

In [ ]:
customers <- read.csv("/kaggle/input/customer-personality-analysis/marketing_campaign.csv", sep = "\t")
customers <- customers[, -1] # remove ID

Data mutation

In [ ]:
customers <- customers %>% 
  mutate(
    # 2014 is used as the "current year" to calculate age 
    # to match the date this dataset is produced
    Age = 2014 - Year_Birth, 
    HaveKids = ifelse(Kidhome + Teenhome > 0, 1, 0),
    # using the last date in "Dt_Customer" to calculate how long the customer 
    # has enrolled as a customer in this supermarket
    DaysCustomer = as.numeric(max(as.Date(Dt_Customer,"%d-%m-%Y"))
                                     - as.Date(Dt_Customer,"%d-%m-%Y")), 
    Married= ifelse(Marital_Status=="Married"|Marital_Status=="Together", 1, 0),
    Graduated= ifelse(Education == "2n Cycle" | Education =="Basic", 0, 1)) %>% 
  select(-c(Year_Birth, Kidhome, Teenhome, Dt_Customer, Marital_Status))

str(customers) # inspect the structure of the data

# 2. Data Preprocessing

## 2.1 Checking for missing value

In [ ]:
sum(is.na(customers))
customers %>% 
  is.na() %>%
  reshape2::melt() %>%
  ggplot(aes(Var2, Var1, fill=value)) + 
    geom_raster() + 
    coord_flip() +
    scale_y_continuous(NULL, expand = c(0, 0)) +
    scale_fill_grey(name = "", 
                    labels = c("Present", 
                               "Missing")) +
    xlab("Observation") +
    theme(axis.text.y  = element_text(size = 4))

There are 24 missing value and they are all in `Income`.

## 2.2 Outlier detection for people attributes

In [ ]:
par(mfrow=c(1, 2))
invisible(lapply(c(2,24), function(i) boxplot(customers[, i], xlab = colnames(customers[i]))))

There are 3 outliers in `Age` and 1 outlier in `Income`. Remove these for further analysis.

In [ ]:
# removing customers whose age is greater than 100
customers <- customers[-which(customers$Age>100),]
# removing customer with highest income
customers <- customers[-which.max(customers$Income),]

Checking the boxplots after removing the outliers.

In [ ]:
par(mfrow=c(1, 2))
invisible(lapply(c(2,24), function(i) boxplot(customers[, i], xlab = colnames(customers[i]))))

There are still some outliers in `Income`, but further observing other data for these customers (see first 7 datapoints in the table below), their `Education` and amounts spent on products do not contradict with their high level of income (e.g. they have high levels of education and have a high spending compared to other customers), so do not consider these as outliers and these datapoints are not removed.


In [ ]:
customers %>% 
  arrange(desc(Income)) %>% 
  select(Education, Income, Age, HaveKids, DaysCustomer, Married,Recency, 
         Complain, MntWines, MntFruits, MntMeatProducts, MntFishProducts, 
         MntSweetProducts, MntGoldProds) %>% 
  head(10)

## 2.3 Attribute and regression matrices
First, create matrices of attributes.

In [ ]:
people_attrs <- customers[, c("Age", "Graduated", "Married", "Income", "HaveKids",
                              "DaysCustomer", "Recency", "Complain")]
product_attrs <- customers[, c("MntWines", "MntFruits", "MntMeatProducts", 
                               "MntFishProducts", "MntSweetProducts", "MntGoldProds")]
promotion_attrs <- customers[, c("NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2", 
                                 "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Response")]
place_attrs <- customers[, c("NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases")]

In [ ]:
attach(customers)

Next, create dataframes for regression analysis. This project will be exploring 3 products with the following response variables:

* Wine
* Food (fish, meat and fruits combined)
* Sweet products

Although spending on gold products are included in the original dataset, this product type will not be explored as common supermarkets will probably not sell gold products.

Our predictors will be all of the customer attributes. Hence, here 3 dataframes are created that contains one of the response variables (wine, food, or sweets) and all of the predictors.

In [ ]:
# wine dataframe
df.wine <- cbind(MntWines, people_attrs)

# Food dataframe
MntFood <- MntFishProducts + MntMeatProducts + MntFruits
df.food <- cbind(MntFood, people_attrs)

# Sweets dataframe
df.sweet <- cbind(MntSweetProducts, people_attrs)

## 2.4 Splitting into training and test set

In [ ]:
# splitting dataframes into training set and test set
set.seed(1)

# wine
wine.split <- initial_split(df.wine, prop=0.7, strata = MntWines)
wine.train <- training(wine.split)
wine.test <- testing(wine.split)

# food
food.split <- initial_split(df.food, prop=0.7, strata = MntFood)
food.train <- training(food.split)
food.test <- testing(food.split)

# sweets
sweet.split <- initial_split(df.sweet, prop=0.7, strata = MntSweetProducts)
sweet.train <- training(sweet.split)
sweet.test <- testing(sweet.split)

## 2.5 Target & Feature engineering

Target & feature engineering include the following steps:

* Log-transformed the outcome variables, which are spending on each product category (`MntWines` for wine, `MntFood` for food, and `MntSweetProducts` for sweets). Set `offset=1` since some spendings are 0 and will return `-Inf` if take log for that. 
* Dummy encoded all categorical vairables (such as `Graduated`, `Married`, `HaveKids`, etc.)
* Centered and scaled all numerical vairables (except for outcome variables)

**Target & Feature engineering for wine dataframe**


In [ ]:
hist(MntWines)

The amount spent on wine `MntWines` is skewed, so log transformation is necessary.

In [ ]:
set.seed(1)
wine.blueprint <- recipe(MntWines ~ ., data = wine.train) %>%
  step_nzv(all_nominal())  %>%
  step_impute_bag(Income) %>% 
  step_log(all_outcomes(), offset = 1) %>% # 
  step_dummy(all_nominal()) %>%
  step_center(all_numeric(), -all_outcomes()) %>%
  step_scale(all_numeric(), -all_outcomes())
prepare.wine <- prep(wine.blueprint, training = wine.train)
baked.wine.train <- recipes::bake(prepare.wine, new_data = wine.train)
baked.wine.test <- recipes::bake(prepare.wine, new_data = wine.test)

Retrieve the mean and standard deviation for `income` after imputation but before centering and scaling, as this will enable reverting the normalized income values back to the original scales.

In [ ]:
set.seed(1)
wine.income.impute <- recipe(MntWines ~ ., data = wine.train) %>%
  step_impute_bag(Income) %>% 
  prep(training = wine.train, retain = TRUE) %>% 
  juice()
income.mean.wine <- mean(wine.income.impute$Income)
income.sd.wine <- sd(wine.income.impute$Income)

**Target & Feature engineering for food dataframe**

In [ ]:
set.seed(1)
food.blueprint <- recipe(MntFood ~ ., data = food.train) %>%
  step_nzv(all_nominal())  %>%
  step_impute_bag(Income) %>% 
  step_log(all_outcomes(), offset = 1) %>% 
  step_dummy(all_nominal()) %>%
  step_center(all_numeric(), -all_outcomes()) %>%
  step_scale(all_numeric(), -all_outcomes())
prepare.food <- prep(food.blueprint, training = food.train)
baked.food.train <- recipes::bake(prepare.food, new_data = food.train)
baked.food.test <- recipes::bake(prepare.food, new_data = food.test)

# retrieving the mean and sd for income
food.income.impute <- recipe(MntFood ~ ., data = food.train) %>%
  step_impute_bag(Income) %>% 
  prep(training = food.train, retain = TRUE) %>% 
  juice()
income.mean.food <- mean(food.income.impute$Income)
income.sd.food <- sd(food.income.impute$Income)

**Target & Feature engineering for sweets dataframe**

In [ ]:
sweet.blueprint <- recipe(MntSweetProducts ~ ., data = sweet.train) %>%
  step_nzv(all_nominal())  %>%
  step_impute_bag(Income) %>% 
  step_log(all_outcomes(), offset = 1) %>% 
  step_dummy(all_nominal()) %>%
  step_center(all_numeric(), -all_outcomes()) %>%
  step_scale(all_numeric(), -all_outcomes())
prepare.sweet <- prep(sweet.blueprint, training = sweet.train)
baked.sweet.train <- recipes::bake(prepare.sweet, new_data = sweet.train)
baked.sweet.test <- recipes::bake(prepare.sweet, new_data = sweet.test)

# retrieving the mean and sd for income
sweet.income.impute <- recipe(MntSweetProducts ~ ., data = sweet.train) %>%
  step_impute_bag(Income) %>% 
  prep(training = sweet.train, retain = TRUE) %>% 
  juice()
income.mean.sweet <- mean(sweet.income.impute$Income)
income.sd.sweet <- sd(sweet.income.impute$Income)

# 3. Method selection

Next, select a method to fit the final models. Workflow is as follows:

* Implement a variety of methods (e.g. Xgboost/random forest/KNN) on only the wine dataframe, and record the training RMSE for each of the methods.
* Based on the training RMSE, select the top models.
* For each of these selected methods, fit the models on the test set, and select the one with the smallest test RMSE as the final model.
* Fit the final model on the data to generate results, i.e. characteristics of customers that are likely to spend more on different product types.

## 3.1 Defining resampling method

In [ ]:
cv <- trainControl(
  method = "repeatedcv",
  number = 10,
  repeats = 5
)

## 3.2 `Most Important` function
Create a function to store the most important features as a dataframe.

In [ ]:
most_important <- function(vip) {
  vip$data %>% 
  pull(Variable) %>% 
  as.data.frame()
}

## 3.3 Linear regression

In [ ]:
wine.lm = train(
  MntWines ~ .,
  data = baked.wine.train,
  trControl = cv,
  method = "glm",
  metric = "RMSE"
)
# storing RMSE on the training set
(rmse.lm <- getTrainPerf(wine.lm)[[1]])

In [ ]:
# storing the four most important features
vip.lm <- most_important(vip(wine.lm))[1:4,]
# variable importance plots
vip(wine.lm)

In [ ]:
## 3.4 KNN

In [ ]:
set.seed(1)

# hypergrid
knn.hyper_grid <- expand.grid(k = seq(2, 26, by=2))

# RMSE as preferred metric
wine.knn <- train(
  MntWines ~ .,
  data = baked.wine.train,
  method = "knn",
  trControl = cv,
  tuneGrid = knn.hyper_grid,
  metric = "RMSE"
)
(rmse.knn <- getTrainPerf(wine.knn)[[1]])

## 3.5 CART
Perform CART and store the training RMSE.

In [ ]:
set.seed(1)
wine_dt1 <- train(
  MntWines ~ .,
  baked.wine.train,
  method = "rpart",
  trControl = cv,
  tuneLength = 20
)
(rmse.cart <- getTrainPerf(wine_dt1)[[1]])

Re-perform CART with the best cp parameter from `wine_dt1`.

In [ ]:
set.seed(1)
wine_dt2 <- rpart(
  MntWines ~ .,
  data    = baked.wine.train,
  method  = "anova",
  control = list(cp = wine_dt1$bestTune[[1]], xval=10)
)

In [ ]:
vip.cart <- most_important(vip(wine_dt2))[1:4,]
vip(wine_dt2)

## 3.6 Random Forest

In [ ]:
# number of features
n_features <- length(setdiff(names(baked.wine.train), "MntWines"))

Perform the baseline Random Forest model.

In [ ]:
# train a default random forest model
wine_rf1 <- ranger(
  MntWines ~ ., 
  data = baked.wine.train,
  mtry = floor(n_features / 3),
  respect.unordered.factors = "order",
  seed = 123
)

# get OOB RMSE
(rmse.rf.default <- sqrt(wine_rf1$prediction.error))

Next, tune hyperparameters for Random Forest.

In [ ]:
# create hyperparameter grid
rf.hyper_grid <- expand.grid(
  mtry = floor(n_features * c(.05, .15, .25, .333, .4)),
  min.node.size = c(1, 3, 5, 10), 
  replace = c(TRUE, FALSE),                               
  sample.fraction = c(.5, .63, .8),                       
  rmse = NA                                               
)

# execute full cartesian grid search
for(i in seq_len(nrow(rf.hyper_grid))) {
  # fit model for ith hyperparameter combination
  fit <- ranger(
    formula         = MntWines ~ ., 
    data            = baked.wine.train, 
    num.trees       = n_features * 10,
    mtry            = rf.hyper_grid$mtry[i],
    min.node.size   = rf.hyper_grid$min.node.size[i],
    replace         = rf.hyper_grid$replace[i],
    sample.fraction = rf.hyper_grid$sample.fraction[i],
    verbose         = FALSE,
    seed            = 123,
    respect.unordered.factors = 'order',
  )
  # export OOB error 
  rf.hyper_grid$rmse[i] <- sqrt(fit$prediction.error)
}

# save the best rmse
rmse.rf.best <- min(rf.hyper_grid$rmse)

# assess top 5 models
rf.hyper_grid %>%
  arrange(rmse) %>%
  mutate(perc_gain = (rmse.rf.default - rmse) / rmse.rf.default * 100) %>%
  head(5)

Rerun the model with impurity-based and permutation-based importance measures.

In [ ]:
# re-run model with impurity-based variable importance
rf_impurity <- ranger(
  formula = MntWines ~ ., 
  data = baked.wine.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 10,
  sample.fraction = .63,
  replace = FALSE,
  importance = "impurity",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

In [ ]:
# re-run model with permutation-based variable importance
rf_permutation <- ranger(
  formula = MntWines ~ ., 
  data = baked.wine.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 10,
  sample.fraction = .63,
  replace = FALSE,
  importance = "permutation",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

Feature interpretation

In [ ]:
rf.p1 <- vip(rf_impurity, bar = TRUE)
rf.p2 <- vip(rf_permutation, bar = TRUE)

vip.rf.impurity <- most_important(rf.p1)[1:4,]
vip.rf.permutation <- most_important(rf.p2)[1:4,]

gridExtra::grid.arrange(rf.p1, rf.p2, nrow = 1)

## 3.7 GBM

In [ ]:
# run a basic GBM model
set.seed(123) 
wine_gbm1 <- gbm(
  formula = MntWines ~ .,
  data = baked.wine.train,
  distribution = "gaussian",  # SSE loss function
  n.trees = 5000,
  shrinkage = 0.1,
  interaction.depth = 3,
  n.minobsinnode = 10,
  cv.folds = 10
)

# find index for number trees with minimum CV error
best <- which.min(wine_gbm1$cv.error)

# get MSE and compute RMSE
(rmse.basic.GBM <- sqrt(wine_gbm1$cv.error[best]))

Feature interpretation

In [ ]:
vip.basic.gbm <- most_important(vip(wine_gbm1))[1:4,]
vip(wine_gbm1)

## 3.8 Stochastic GBM

In [ ]:
# initiate h2o session
h2o.no_progress()
h2o.init()

# convert training data to h2o object
wine.train_h2o <- as.h2o(baked.wine.train)

# set the response column to Sale_Price
response <- "MntWines"

# set the predictor names
predictors <- setdiff(colnames(baked.wine.train), response)

Tune stochastic GBM hyperparameters.

In [ ]:
# refined hyperparameter grid
sto.gbm.hyper_grid <- list(
  sample_rate = c(0.5, 0.75, 1),              # row subsampling
  col_sample_rate = c(0.5, 0.75, 1),          # col subsampling for each split
  col_sample_rate_per_tree = c(0.5, 0.75, 1)  # col subsampling for each tree
)

# random grid search strategy
search_criteria <- list(
  strategy = "RandomDiscrete",
  stopping_metric = "mse",
  stopping_tolerance = 0.001,
  stopping_rounds = 10,
  max_runtime_secs = 60*10
)

# perform grid search
grid <- h2o.grid(
  algorithm = "gbm",
  grid_id = "gbm_grid",
  x = predictors,
  y = response,
  training_frame = wine.train_h2o,
  hyper_params = sto.gbm.hyper_grid,
  ntrees = 6000,
  learn_rate = 0.01,
  max_depth = 7,
  min_rows = 5,
  nfolds = 10,
  stopping_rounds = 10,
  stopping_tolerance = 0,
  search_criteria = search_criteria,
  seed = 123
)

# collect the results and sort by the model performance metric of choice
grid_perf <- h2o.getGrid(
  grid_id = "gbm_grid",
  sort_by = "mse",
  decreasing = FALSE
)

grid_perf

In [ ]:
best_model_id <- grid_perf@model_ids[[1]]
best_model <- h2o.getModel(best_model_id)

# Now let’s get performance metrics on the best model
h2o.performance(model = best_model, xval = TRUE)

Saving and retrieving RMSE for stochastic GBM

In [ ]:
(rmse.stochatic.gbm <- sqrt(grid_perf@summary_table[[5]][1]))

Feature interpretation

In [ ]:
vip.stochastic.gbm <- most_important(vip(best_model))[1:4,]
vip(best_model)

## 3.9 XGBoost
All categorical variables were already encoded numerically. Next, convert the training data frame to matrices

In [ ]:
wine.X <- as.matrix(baked.wine.train[setdiff(names(baked.wine.train), "MntWines")])
wine.Y <- baked.wine.train$MntWines

Perform grid search.

In [ ]:
set.seed(1)
wine_xgb <- xgb.cv(
  data = wine.X,
  label = wine.Y,
  nrounds = 6000,
  objective = "reg:linear",
  early_stopping_rounds = 50, 
  nfold = 10,
  params = list(
    eta = 0.1,
    max_depth = 3,
    min_child_weight = 3,
    subsample = 0.8,
    colsample_bytree = 1.0),
  verbose = 0
)  

In [ ]:
# minimum test CV RMSE
(rmse.xgb.baseline <- min(wine_xgb$evaluation_log$test_rmse_mean))

Next, perform a grid search that examines various regularization parameters

In [ ]:
# the below code is commented out due to long processing time and warning messages, the results were shown in the next cell
# wine.xbg.hyper_grid <- expand.grid(
#   eta = 0.01,
#   max_depth = 5, 
#   min_child_weight = 3,
#   subsample = 0.5, 
#   colsample_bytree = 0.5,
#   gamma = c(0, 1, 10, 100, 1000),
#   lambda = c(0, 1e-2, 0.1, 1, 100),
#   alpha = c(0, 1e-2, 0.1, 1, 100),
#   rmse = 0,          # a place to dump RMSE results
#   trees = 0          # a place to dump required number of trees
# )

# # grid search
# for(i in seq_len(nrow(wine.xbg.hyper_grid))) {
#   set.seed(1)
#   m <- xgb.cv(
#     data = wine.X,
#     label = wine.Y,
#     nrounds = 4000,
#     objective = "reg:linear",
#     early_stopping_rounds = 50, 
#     nfold = 10,
#     verbose = FALSE,
#     params = list( 
#       eta = wine.xbg.hyper_grid$eta[i], 
#       max_depth = wine.xbg.hyper_grid$max_depth[i],
#       min_child_weight = wine.xbg.hyper_grid$min_child_weight[i],
#       subsample = wine.xbg.hyper_grid$subsample[i],
#       colsample_bytree = wine.xbg.hyper_grid$colsample_bytree[i],
#       gamma = wine.xbg.hyper_grid$gamma[i], 
#       lambda = wine.xbg.hyper_grid$lambda[i], 
#       alpha = wine.xbg.hyper_grid$alpha[i]
#     ) 
#   )
#   wine.xbg.hyper_grid$rmse[i] <- min(m$evaluation_log$test_rmse_mean)
#   wine.xbg.hyper_grid$trees[i] <- m$best_iteration
# }


Grid search results:

In [ ]:
# results
rmse.xgb.best <- min(wine.xbg.hyper_grid$rmse)

wine.xbg.hyper_grid %>%
  filter(rmse > 0) %>%
  arrange(rmse) %>%
  glimpse()

## 3.10 method comparison


Below is a table summarising the RMSE for training set for each method, in descending order.

In [ ]:
reg.methods <- c("Basic GBM", "KNN", "LM", 
                 "CART","RF","RF (baseline)", 
                 "Stochastic GBM",
                 "Xgboost")
reg.rmses <- c(rmse.basic.GBM, rmse.knn, rmse.lm, 
               rmse.cart,rmse.rf.best, rmse.rf.default, 
               rmse.stochatic.gbm,
               rmse.xgb.best)
df.rmse <- data.frame(reg.methods, reg.rmses)
df.rmse %>% 
  arrange(reg.rmses)

From the above table, 3 models that have a lower RMSE are chosen:

* `Xgboost` (chosen as a representation of the Gradient Boosting methods)
* RF (Random Forest)
* CART

# 4. Model and Regression Trees

## 4.1 CART
Plot the resulted tree from CART model `wine_dt2`.

In [ ]:
rpart.plot(wine_dt2)

*Note: the outputted plot and the plot on which the below analysis below is based are different.*
*The difference however does not result in difference in the next steps and conclusions.*

The node with the highest predicted value is the node on the far right with predicted `y = 6.4`. Following the branches from root node to this terminal node, the following characteristics are summarized:

In [ ]:
print(paste("highest prediction:", exp(6.4)))
print(paste("percentile:", sum(customers$MntWines<exp(6.4))/length(customers$MntWines)))
print(paste("income >=:", -0.33*income.sd.wine+income.mean.wine))
print(paste("income >:", 0.15*income.sd.wine+income.mean.wine))
print(paste("income <:", 2.6*income.sd.wine+income.mean.wine))
print(paste("income >=:", 0.48*income.sd.wine+income.mean.wine))
print(paste("days customers >:", 0.24*sd(customers$DaysCustomer)+mean(customers$DaysCustomer)))

Make predictions on the test set and save the resulted RMSE.

In [ ]:
wine.cart.pred <- predict(wine_dt2, newdata = baked.wine.test[, -9])
(test.rmse.wine.cart <- rmse(as.numeric(unlist(baked.wine.test[, 9])), wine.cart.pred))

## 4.2 Xgboost
Retrieve the list of optimal hyperparameters from above to train the final model.

In [ ]:
wine.xgb.params <- list(
  eta = 0.01,
  max_depth = 5,
  min_child_weight = 3,
  subsample = 0.5,
  colsample_bytree = 0.5,
  gamma = 1,
  alpha = 1,
  lambda = 100
)

In [ ]:
# train final model
set.seed(1)
wine.xgb.fit.final <- xgboost::xgboost(
  params = wine.xgb.params,
  data = wine.X,
  label = wine.Y,
  nrounds = 1343,
  objective = "reg:squarederror",
  verbose = FALSE)

In [ ]:
gr <- xgb.plot.tree(model=wine.xgb.fit.final, trees=1342, render=FALSE)
export_graph(gr, 'tree.pdf', width=1500, height=1900)
xgb.plot.tree(model = wine.xgb.fit.final, trees = 1342)

In [ ]:
xgbtable <- xgb.model.dt.tree(model=wine.xgb.fit.final)
xgbtable %>% 
  filter(Tree==1342 & Feature == "Leaf") %>% 
  arrange(desc(Quality))

The node that gives the highest prediction is node 8 (highest `quality` in the above table). Following the branches from root node to this terminal node, the following characteristics are summarized:

In [ ]:
xgbtable %>% 
  filter(Tree==1342)

In [ ]:
print(paste("(node 0->1) income <:", 0.9065892*income.sd.wine+income.mean.wine))
print(paste("(node 1->3) income <:", -0.7366908*sd(customers$Age)+mean(customers$Age)))
print(paste("(node 3->8) income >:", -1.3820734*sd(customers$Recency)+mean(customers$Recency)))

In [ ]:
wine.xgb.pred <- predict(wine.xgb.fit.final, newdata = data.matrix(baked.wine.test[, -9]))
(test.rmse.wine.xgb <- rmse(as.numeric(unlist(baked.wine.test[, 9])), wine.xgb.pred))

In [ ]:
vip.xgb <- most_important(vip(wine.xgb.fit.final))[1:4,]
vip(wine.xgb.fit.final)

## 4.3 Random Forest
Plotting the tree using the tuned parameters found from the previous section.

In [ ]:
set.seed(1)
wine.cf <- party::cforest(MntWines ~ ., 
                          data=baked.wine.train, 
                          controls=cforest_control(mtry=3, 
                                                   mincriterion = 1,
                                                   replace = FALSE, ntree = 2000, 
                                                   fraction = 0.63))
wine.pt <- prettytree(wine.cf@ensemble[[1]], names(wine.cf@data@get("input"))) 
wine.nt <- new("BinaryTree") 
wine.nt@tree <- wine.pt 
wine.nt@data <- wine.cf@data 
wine.nt@responses <- wine.cf@responses

plot(wine.nt, type="simple")

*Note: the outcome is slightly different from the one on which the following analysis is based*

The highest node is node 15 with predicted `y = 6.81`. Following the branches from root node to this terminal node, summarise the following characteristics are summarized:

In [ ]:
print(paste("highest prediction:", exp(6.81)))
print(paste("percentile:", sum(customers$MntWines<exp(6.81))/length(customers$MntWines)))
print(paste("married <=:", -1.352*sd(customers$HaveKids)+mean(customers$HaveKids)))
print(paste("days customers >:", 0.936*sd(customers$DaysCustomer)+mean(customers$DaysCustomer)))
print(paste("income >=:", 0.389*income.sd.wine+income.mean.wine))
print(paste("income >:", -0.467*income.sd.wine+income.mean.wine))
print(paste("age >:", -0.02*sd(customers$Age)+mean(customers$Age)))
print(paste("kids <=:", -1.561*sd(customers$HaveKids)+mean(customers$HaveKids)))

Save the test error for Random Forest.

In [ ]:
wine.rf.pred <- predict(rf_impurity, data=baked.wine.test[, -9], type="response")
(test.rmse.wine.rf <- rmse(as.numeric(unlist(baked.wine.test[, 9])), wine.rf.pred$predictions))

## 4.4 Method comparison

In [ ]:
test.methods <- c("Xgboost", "CART", "RF")
test.rmses <- c(test.rmse.wine.xgb,test.rmse.wine.cart, test.rmse.wine.rf)
df.test.rmse <- data.frame(test.methods, test.rmses)
df.test.rmse %>% 
  arrange(test.rmses)

Amongst the three methods, Random Forest has the lowest RMSE on the test set. On the merit of this, Random Forest is chosen to fit the final models.

# 5 Final model and predictions

## 5.1 Amount spent on wine (see above)

In [ ]:
## 5.2 Amount spent on food (fish, meat, fruits)

Tuning hyperparameters

In [ ]:
# create hyperparameter grid
food.rf.hyper_grid <- expand.grid(
  mtry = floor(n_features * c(.05, .15, .25, .333, .4)),
  min.node.size = c(1, 3, 5, 10), 
  replace = c(TRUE, FALSE),                               
  sample.fraction = c(.5, .63, .8),                       
  rmse = NA                                               
)

# execute full cartesian grid search
for(i in seq_len(nrow(food.rf.hyper_grid))) {
  # fit model for ith hyperparameter combination
  fit <- ranger(
    formula         = MntFood ~ ., 
    data            = baked.food.train, 
    num.trees       = n_features * 10,
    mtry            = food.rf.hyper_grid$mtry[i],
    min.node.size   = food.rf.hyper_grid$min.node.size[i],
    replace         = food.rf.hyper_grid$replace[i],
    sample.fraction = food.rf.hyper_grid$sample.fraction[i],
    verbose         = FALSE,
    seed            = 123,
    respect.unordered.factors = 'order',
  )
  # export OOB error 
  food.rf.hyper_grid$rmse[i] <- sqrt(fit$prediction.error)
}

# assess top 5 models
food.rf.hyper_grid %>%
  arrange(rmse) %>%
  mutate(perc_gain = (rmse.rf.default - rmse) / rmse.rf.default * 100) %>%
  head(5)

Rerun models with best parameters and feature interpretation

In [ ]:
# re-run model with impurity-based variable importance
food.rf_impurity <- ranger(
  formula = MntFood ~ ., 
  data = baked.food.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 3,
  sample.fraction = .50,
  replace = FALSE,
  importance = "impurity",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

# re-run model with permutation-based variable importance
food.rf_permutation <- ranger(
  formula = MntFood ~ ., 
  data = baked.food.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 3,
  sample.fraction = .50,
  replace = FALSE,
  importance = "permutation",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

food.rf.p1 <- vip::vip(food.rf_impurity)
food.rf.p2 <- vip::vip(food.rf_permutation)

gridExtra::grid.arrange(food.rf.p1, food.rf.p2, nrow = 1)

`Income`,  `HaveKids`, `DaysCustomer` and `Age` are the four most important features.

Plotting the resulted tree:

In [ ]:
# library("party")
set.seed(1)
food.cf <- party::cforest(MntFood ~ ., data=baked.food.train,
                     controls=cforest_control(
                       mtry=3, 
                       replace = FALSE, 
                       ntree = 2000, 
                       fraction = 0.5))


food.pt <- prettytree(food.cf@ensemble[[1]], names(food.cf@data@get("input"))) 
food.nt <- new("BinaryTree") 
food.nt@tree <- food.pt 
food.nt@data <- food.cf@data 
food.nt@responses <- food.cf@responses 

plot(food.nt, type="simple")

The highest node is node 19 with predicted `y = 6.618`. Following the branches from root node to this terminal node, the following characteristics are summarized:

In [ ]:
print(paste("highest prediction:", exp(6.618)))
print(paste("percentile:", sum(df.food$MntFood<exp(6.618))/length(df.food$MntFood)))
print(paste("age <=:", 1.09*sd(customers$Age)+mean(customers$Age)))
print(paste("days customers >:", -0.402*sd(customers$DaysCustomer)+mean(customers$DaysCustomer)))
print(paste("have kids <=:", -1.611*sd(customers$HaveKids)+mean(customers$HaveKids)))
print(paste("age >:", -1.013*sd(customers$Age)+mean(customers$Age)))
print(paste("income >:", 0.261*income.sd.food+income.mean.food))

## 5.3 Amount spent on sweets

Tuning hyperparameters

In [ ]:
# create hyperparameter grid
sweet.rf.hyper_grid <- expand.grid(
  mtry = floor(n_features * c(.05, .15, .25, .333, .4)),
  min.node.size = c(1, 3, 5, 10), 
  replace = c(TRUE, FALSE),                               
  sample.fraction = c(.5, .63, .8),                       
  rmse = NA                                               
)

# execute full cartesian grid search
for(i in seq_len(nrow(sweet.rf.hyper_grid))) {
  # fit model for ith hyperparameter combination
  fit <- ranger(
    formula         = MntSweetProducts ~ ., 
    data            = baked.sweet.train, 
    num.trees       = n_features * 10,
    mtry            = sweet.rf.hyper_grid$mtry[i],
    min.node.size   = sweet.rf.hyper_grid$min.node.size[i],
    replace         = sweet.rf.hyper_grid$replace[i],
    sample.fraction = sweet.rf.hyper_grid$sample.fraction[i],
    verbose         = FALSE,
    seed            = 123,
    respect.unordered.factors = 'order',
  )
  # export OOB error 
  sweet.rf.hyper_grid$rmse[i] <- sqrt(fit$prediction.error)
}

# assess top 5 models
sweet.rf.hyper_grid %>%
  arrange(rmse) %>% 
  head(5)

Rerun models with best parameters and feature interpretation

In [ ]:
# re-run model with impurity-based variable importance
sweet.rf_impurity <- ranger(
  formula = MntSweetProducts ~ ., 
  data = baked.sweet.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 1,
  sample.fraction = .63,
  replace = TRUE,
  importance = "impurity",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

# re-run model with permutation-based variable importance
sweet.rf_permutation <- ranger(
  formula = MntSweetProducts ~ ., 
  data = baked.sweet.train, 
  num.trees = 2000,
  mtry = 3,
  min.node.size = 1,
  sample.fraction = .63,
  replace = TRUE,
  importance = "permutation",
  respect.unordered.factors = "order",
  verbose = FALSE,
  seed  = 123
)

sweet.rf.p1 <- vip::vip(sweet.rf_impurity)
sweet.rf.p2 <- vip::vip(sweet.rf_permutation)

gridExtra::grid.arrange(sweet.rf.p1, sweet.rf.p2, nrow = 1)

`Income` is the most important feature; relative importance for other variables differ.

Plotting the resulted tree

In [ ]:
# library("party")
set.seed(1)
sweet.cf <- party::cforest(MntSweetProducts ~ ., data=baked.sweet.train,
                     controls=cforest_control(
                       mtry=3, 
                       replace = TRUE, 
                       ntree = 2000, 
                       fraction = 0.63))

sweet.pt <- prettytree(sweet.cf@ensemble[[1]], names(sweet.cf@data@get("input"))) 
sweet.nt <- new("BinaryTree") 
sweet.nt@tree <- sweet.pt 
sweet.nt@data <- sweet.cf@data 
sweet.nt@responses <- sweet.cf@responses 

plot(sweet.nt, type="simple")

The highest node is node 22 with predicted `y = 4.281`. Following the branches from root node to this terminal node, the following characteristics are summarized:

In [ ]:
print(paste("highest prediction:", exp(4.281)))
print(paste("percentile:", sum(df.sweet$MntSweetProducts<exp(4.281))/length(df.sweet$MntSweetProducts)))
print(paste("income >:", 1.291*income.sd.sweet+income.mean.sweet))
print(paste("income >:", 0.648*income.sd.sweet+income.mean.sweet))
print(paste("graduated >:", -2.728*sd(customers$Graduated)+mean(customers$Graduated)))
print(paste("days customers <=:", -0.978*sd(customers$DaysCustomer)+mean(customers$DaysCustomer)))

# 6. Conclusion and insights


## 6.1 Feature importance

In [ ]:
importance <- c("1st", "2nd", "3rd", "4th")
df.vip <- data.frame(
  importance,
  vip.basic.gbm,
  vip.cart,
  vip.lm,
  vip.rf.impurity,
  vip.stochastic.gbm,
  vip.xgb
)

df.vip %>% 
  mutate_all(funs(str_replace(., "DaysCustomer", "Days_Customer")))

It can be seen from the first and second rows of the above table that:

* `Income` is always the most important variable in predicting spending
* `DaysCustomer` (or `Days_Customer` used in presentation) is the next most important variable
* other variables such as `Age`, `Recency` and `Graduated` also have some impacts

## 6.2 Recommendations actions
Based on the above findings, it is recommended to target customers that are 

1. in the high income group (e.g. with an income higher than $57,000), and 
2. old customers (e.g. who have been registered with this supermarket for more than 270 days)

The supermarket may consider various marketing strategies including: 
1. Forming VIP clubs for high income groups; 
2. creating loyalty/rewards programmes for old customers; and 
3. send out offers for each product targeting specifically at the “ideal customer” identified for each product category.